In [1]:
# Example notebook for loading and evaluating robust model.
import numpy as np
import torch as ch
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook

from robustness.model_utils import make_and_restore_model
from robustness.tools.label_maps import CLASS_DICT
from robustness.datasets import CIFAR
ds = CIFAR('./data/cifar/')

In [2]:
NUM_WORKERS = 2
BATCH_SIZE  = 500

train_loader, test_loader = ds.make_loaders(workers=NUM_WORKERS, batch_size=BATCH_SIZE, shuffle_val=True)
_, (im, label) = next(enumerate(test_loader))

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# load model
model, _ = make_and_restore_model(arch='resnet18', dataset=ds, resume_path='./models/cifar_Linf_FE_N3.pt')
model.cuda()
model.eval()

=> loading checkpoint './models/cifar_Linf_FE_N3.pt'
=> loaded checkpoint './models/cifar_Linf_FE_N3.pt' (epoch 61)


AttackerModel(
  (normalizer): InputNormalize()
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): SequentialWithArgs(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1

In [4]:
# Generate adversarial attacks by loss maximization
ATTACK_EPS      = 8/255.0
ATTACK_STEPS    = 20
ATTACK_STEPSIZE = 1.5*ATTACK_EPS/ATTACK_STEPS

kwargs = {
    'constraint':'inf',
    'eps': ATTACK_EPS,
    'step_size': ATTACK_STEPSIZE,
    'iterations': ATTACK_STEPS,
    'do_tqdm': True,
}

_, im_adv = model(im.cuda(), label.cuda(), make_adv=True, **kwargs)

Current loss: 1.3480368852615356: 100%|██████████| 20/20 [00:04<00:00,  6.78it/s]


In [5]:
# Get predicted labels for clean examples
logits, _ = model(im.cuda())
label_pred = ch.argmax(logits, dim=1)

# Get predicted labels for adversarial examples
logits_adv, _ = model(im_adv.cuda())
label_pred_adv = ch.argmax(logits_adv, dim=1)

# compute clean and adversarial accuracy over this set
acc = np.sum((label==label_pred.cpu()).numpy())/BATCH_SIZE
adv_acc = np.sum((label==label_pred_adv.cpu()).numpy())/BATCH_SIZE
print('Clean accuracy = %.2f | Adversarial accuracy = %.2f' % (acc*100, adv_acc*100))

Clean accuracy = 81.20 | Adversarial accuracy = 51.80
